<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/hak_1_KoBERT_dialect_dataset_label_generation_kw_70000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000_std.pt'

#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=60fa8539e66ab0656eb43a6fb94051ab8b78333d2cbf4efe8327ea1e919649c8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 3.0 MB 33.1 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 


#KoBERT 다운로드(사용)#

In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2yuxuj1o
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2yuxuj1o
     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 24.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.1 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 61 kB 522 kB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=b704c748d2baed2071a6386b2f2995a72b7cbf495326d35eecbce0e705b98470
  Stored in directory: /tmp/pip-ephem-wheel-cache-t16kvif1/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#KoBERT 입력 데이터로 변환(사용)#

In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#원본 데이터셋으로부터 데이터 추출하기#

In [11]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [12]:
!echo $VALID_DIR

/content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/valid


In [13]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep json | grep -v data | grep -v answer| grep -v config').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/train
/content
popd -> /content


In [14]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep json | grep -v data | grep -v answer | grep -v config').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid
/content
popd -> /content


In [15]:
len(train_json_list)

4717

In [16]:
len(valid_json_list)

828

#추출한 데이터에 감정 레이블 추가(사용)#

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [18]:
#모델 불러오기
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [29]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [20]:
import csv

def GenLabelFromData70000(path, jsonfilename):
  try:
    fin = open('%s/%s.data' % (path, jsonfilename), 'r')
  except:
    print('No file(%s/%s.data)' % (path, jsonfilename))
    return

  with open('%s/%s.data' % (path, jsonfilename), 'r') as f:
    with open('%s/%s.answer70000' % (path, jsonfilename), 'w') as o:
      csv_data = csv.reader(f, delimiter='\t')
      for i, line in enumerate(csv_data):
        std = line[0]
        dia = line[1]
        idx, val = predict(line[0])
        #print('%s(%d) %s\n' % (val, idx, std))
        #print('[%d]%s/%s/%d/%s\n' % (i, std, dia, idx, val))
        o.write('%s\t%s\t%d\n' % (std, dia, idx))
  print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [21]:
train_json_list[0:171]

['DGDQ20000020.json',
 'DGDQ20000027.json',
 'DGDQ20000029.json',
 'DGDQ20000032.json',
 'DGDQ20000034.json',
 'DGDQ20000036.json',
 'DGDQ20000045.json',
 'DGDQ20000047.json',
 'DGDQ20000053.json',
 'DGDQ20000061.json',
 'DGDQ20000064.json',
 'DGDQ20000067.json',
 'DGDQ20000068.json',
 'DGDQ20000070.json',
 'DGDQ20000071.json',
 'DGDQ20000072.json',
 'DGDQ20000077.json',
 'DGDQ20000078.json',
 'DGDQ20000082.json',
 'DGDQ20000083.json',
 'DGDQ20000084.json',
 'DGDQ20000088.json',
 'DGDQ20000089.json',
 'DGDQ20000090.json',
 'DGDQ20000094.json',
 'DGDQ20000098.json',
 'DGDQ20000099.json',
 'DGDQ20000102.json',
 'DGDQ20000103.json',
 'DGDQ20000106.json',
 'DGDQ20000109.json',
 'DGDQ20000110.json',
 'DGDQ20000111.json',
 'DGDQ20000115.json',
 'DGDQ20000119.json',
 'DGDQ20000120.json',
 'DGDQ20000121.json',
 'DGDQ20000122.json',
 'DGDQ20000123.json',
 'DGDQ20000124.json',
 'DGDQ20000125.json',
 'DGDQ20000131.json',
 'DGDQ20000132.json',
 'DGDQ20000135.json',
 'DGDQ20000138.json',
 'DGDQ2000

In [22]:
valid_json_list[0:58]

['DGIN21500168.json',
 'DGIN21500169.json',
 'DGIN21500170.json',
 'DGIN21500171.json',
 'DGIN21500172.json',
 'DGIN21500173.json',
 'DGIN21500174.json',
 'DGIN21500175.json',
 'DGIN21500176.json',
 'DGIN21500177.json',
 'DGIN21500179.json',
 'DGIN21500180.json',
 'DGIN21500181.json',
 'DGIN21500182.json',
 'DGIN21500183.json',
 'DGIN21500184.json',
 'DGIN21500185.json',
 'DGIN21500186.json',
 'DGIN21500187.json',
 'DGIN21500188.json',
 'DGIN21500189.json',
 'DGIN21500190.json',
 'DGIN21500191.json',
 'DGIN21500192.json',
 'DGIN21500193.json',
 'DGIN21500194.json',
 'DGIN21500195.json',
 'DGIN21500196.json',
 'DGIN21500197.json',
 'DGIN21500198.json',
 'DGIN21500199.json',
 'DGIN21500200.json',
 'DGIN21500201.json',
 'DGIN21500202.json',
 'DGIN21500203.json',
 'DGIN21500204.json',
 'DGIN21500205.json',
 'DGIN21500206.json',
 'DGIN21500207.json',
 'DGIN21500208.json',
 'DGIN21500209.json',
 'DGIN21500210.json',
 'DGIN21500211.json',
 'DGIN21500212.json',
 'DGIN21500213.json',
 'DGIN2150

In [27]:
# for문을 돌면서 json 파일 내에 있는 표준어 문장과 사투리 문장을 추출해서 FILENAME.data로 저장
import json

def GenDataFromJson(path, jsonfilename):
  with open('%s/%s' % (path, jsonfilename), 'r') as f:
    json_data = json.load(f);
  #print(json.dumps(json_data, indent='\t'))
  ut = json_data['utterance']

  with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
    for u in ut:
      f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))
   
%pushd $TRAIN_DIR
# 450개 파일 * 350개 문장
#for f in train_json_list[0:171]:
  #GenDataFromJson(TRAIN_DIR, f)
%popd

%pushd $VALID_DIR
# 150개 파일 350개 문장
#for f in valid_json_list[0:58]:
  #GenDataFromJson(VALID_DIR, f)
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/train
/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid
popd -> /content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid
/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid
/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid
popd -> /content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/valid


In [ ]:
#%pushd $TRAIN_DIR
## 450개 파일 350개 문장
#for i, f in enumerate(train_json_list[0:100]):
#  print('[%d][%s]' % (i, f))
#  GenLabelFromData70000(TRAIN_DIR, f)
#%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_kw_labeled/train
[0][DGDQ20000020.json]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000020.json.answer

[1][DGDQ20000027.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000027.json.answer

[2][DGDQ20000029.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000029.json.answer

[3][DGDQ20000032.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000032.json.answer

[4][DGDQ20000034.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000034.json.answer

[5][DGDQ20000036.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000036.json.answer

[6][DGDQ20000045.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000045.json.answer

[7][DGDQ20000047.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train/DGDQ20000047.json.answer

[8][DGDQ20000053.json]
>> Gener

In [ ]:
%pushd $TRAIN_DIR
# 450개 파일 350개 문장
for i, f in enumerate(train_json_list[17:100]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(TRAIN_DIR, f)
%popd

In [ ]:
%pushd $TRAIN_DIR
# 450개 파일 350개 문장
for i, f in enumerate(train_json_list[101:171]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(TRAIN_DIR, f)
%popd

In [ ]:
%pushd $VALID_DIR
# 450개 파일 350개 문장
for i, f in enumerate(valid_json_list[0:58]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(VALID_DIR, f)
%popd